# 10_11_분석기획
## 1. 액티브 / 휴폐업 데이터 산업코드 기준으로 대분류 및 소분류 카테고리 추가
## 2. 산업분류를 이용하여 업종별 라벨링 후 시가 총액 가장 높은 5개 종목 크롤링

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
import seaborn as sns
# 한글
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm , rc
font = fm.FontProperties(fname='C:\Windows\Fonts/gulim.ttc').get_name()
#셀레니움
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time


In [2]:
nb_codzip = pd.read_excel('넘블 챌린지 개방데이터 코드집.xlsx', sheet_name=0)
nb_codzip_finance = pd.read_excel('넘블 챌린지 개방데이터 코드집.xlsx', sheet_name=1)
nb_layout = pd.read_excel('넘블 챌린지 개방데이터 레이아웃.xlsx')
nb_active_out = pd.read_excel('액티브 중소법인 재무보유.xlsx', sheet_name=0)
nb_active_overview = pd.read_excel('액티브 중소법인 재무보유.xlsx', sheet_name=1)
nb_active_oclog = pd.read_excel('액티브 중소법인 재무보유.xlsx', sheet_name=2)
nb_deactive_overview = pd.read_excel('휴폐업 중소법인 재무보유.xlsx', sheet_name=0)
nb_deactive_oclog = pd.read_excel('휴폐업 중소법인 재무보유.xlsx', sheet_name=1)
nb_finance_data = pd.read_csv('재무데이터.txt', encoding='cp949', sep='\t')


In [3]:
nb_active_ind = pd.read_excel('액티브_중소법인_기업개요_산업분류.xlsx')
nb_deactive_ind = pd.read_excel('휴폐업_중소법인_기업개요_산업분류.xlsx')

## 1. 액티브 / 휴폐업 데이터 산업코드 기준으로 대분류 및 소분류 카테고리 추가
-> 한국표준산업분류 기준으로 산업코드 대분류 / 소분류 추가

In [73]:
nb_active_ind.ind_01_cate.unique()

array(['건설업', '도매 및 소매업', '제조업', '정보통신업', '사업시설 관리, 사업 지원 및 임대 서비스업',
       '운수 및 창고업', '전문, 과학 및 기술 서비스업', '숙박 및 음식점업', '부동산업',
       '수도, 하수 및 폐기물 처리, 원료 재생업', '금융 및 보험업', '예술, 스포츠 및 여가관련 서비스업',
       '협회 및 단체, 수리 및 기타 개인 서비스업', '공공 행정, 국방 및 사회보장 행정', '교육 서비스업',
       '전기, 가스, 증기 및 공기 조절 공급업', '보건업 및 사회복지 서비스업', nan,
       '가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동', '국제 및 외국기관'], dtype=object)

In [74]:

ind_01_dict = {}
for ind in ['01', '02', '03']:
    ind_01_dict[ind] = '농업, 임업 및 어업'
for ind in ['05', '06', '07', '08'] : 
    ind_01_dict[ind] = '광업'
for ind in range(10, 35):
    ind_01_dict[str(ind)] = '제조업'
for ind in range(35, 36):
    ind_01_dict[str(ind)] = '전기, 가스, 증기 및 공기 조절 공급업'
for ind in range(36, 40):
    ind_01_dict[str(ind)] = '수도, 하수 및 폐기물 처리, 원료 재생업'
for ind in range(41, 43):
    ind_01_dict[str(ind)] = '건설업'
for ind in range(45, 48):
    ind_01_dict[str(ind)] = '도매 및 소매업'
for ind in range(49, 53):
    ind_01_dict[str(ind)] = '운수 및 창고업'
for ind in range(55, 57):
    ind_01_dict[str(ind)] = '숙박 및 음식점업'
for ind in range(58, 64):
    ind_01_dict[str(ind)] = '정보통신업'
for ind in range(64, 67):
    ind_01_dict[str(ind)] = '금융 및 보험업'
for ind in range(68, 69):
    ind_01_dict[str(ind)] = '부동산업'
for ind in range(70, 74):
    ind_01_dict[str(ind)] = '전문, 과학 및 기술 서비스업'
for ind in range(74, 77):
    ind_01_dict[str(ind)] = '사업시설 관리, 사업 지원 및 임대 서비스업'
for ind in range(84, 85):
    ind_01_dict[str(ind)] = '공공 행정, 국방 및 사회보장 행정'
for ind in range(85, 86):
    ind_01_dict[str(ind)] = '교육 서비스업'
for ind in range(86, 88):
    ind_01_dict[str(ind)] = '보건업 및 사회복지 서비스업'
for ind in range(90, 92):
    ind_01_dict[str(ind)] = '예술, 스포츠 및 여가관련 서비스업'
for ind in range(94, 97):
    ind_01_dict[str(ind)] = '협회 및 단체, 수리 및 기타 개인 서비스업'
for ind in range(97, 99):
    ind_01_dict[str(ind)] = '가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동'
for ind in range(99, 100):
    ind_01_dict[str(ind)] = '국제 및 외국기관'


In [75]:
ind_02_dict = {'01' : '농업',
'02' : '임업',
'03' : '어업',
'05': '석탄, 원유 및 천연가스 광업',
'06': '금속 광업',
'07': '비금속광물 광업; 연료용 제외',
'08': '광업 지원 서비스업',
'10': '식료품 제조업',
'11': '음료 제조업',
'12': '담배 제조업',
'13': '섬유제품 제조업; 의복 제외',
'14': '의복, 의복 액세서리 및 모피제품 제조업',
'15': '가죽, 가방 및 신발 제조업',
'16': '목재 및 나무제품 제조업; 가구 제외',
'17': '펄프, 종이 및 종이제품 제조업',
'18': '인쇄 및 기록매체 복제업',
'19': '코크스, 연탄 및 석유정제품 제조업',
'20': '화학 물질 및 화학제품 제조업; 의약품 제외',
'21': '의료용 물질 및 의약품 제조업',
'22': '고무 및 플라스틱제품 제조업',
'23': '비금속 광물제품 제조업',
'24': '1차 금속 제조업',
'25': '금속 가공제품 제조업; 기계 및 가구 제외',
'26': '전자 부품, 컴퓨터, 영상, 음향 및 통신장비 제조업',
'27': '의료, 정밀, 광학 기기 및 시계 제조업',
'28': '전기장비 제조업',
'29': '기타 기계 및 장비 제조업',
'30': '자동차 및 트레일러 제조업',
'31': '기타 운송장비 제조업',
'32': '가구 제조업',
'33': '기타 제품 제조업',
'34': '산업용 기계 및 장비 수리업',
'35': '전기, 가스, 증기 및 공기 조절 공급업',
'36': '수도업',
'37': '하수, 폐수 및 분뇨 처리업',
'38': '폐기물 수집, 운반, 처리 및 원료 재생업',
'39': '환경 정화 및 복원업',
'41': '종합 건설업',
'42': '전문직별 공사업',
'45': '자동차 및 부품 판매업',
'46': '도매 및 상품 중개업',
'47': '소매업; 자동차 제외',
'49': '육상 운송 및 파이프라인 운송업',
'50': '수상 운송업',
'51': '항공 운송업',
'52': '창고 및 운송관련 서비스업',
'55': '숙박업',
'56': '음식점 및 주점업',
'58': '출판업',
'59': '영상․오디오 기록물 제작 및 배급업',
'60': '방송업',
'61': '우편 및 통신업',
'62': '컴퓨터 프로그래밍, 시스템 통합 및 관리업',
'63': '정보서비스업',
'64': '금융업',
'65': '보험 및 연금업',
'66': '금융 및 보험관련 서비스업',
'68': '부동산업',
'70': '연구개발업',
'71': '전문 서비스업',
'72': '건축 기술, 엔지니어링 및 기타 과학기술 서비스업',
'73': '기타 전문, 과학 및 기술 서비스업',
'74': '사업시설 관리 및 조경 서비스업',
'75': '사업 지원 서비스업',
'76': '임대업; 부동산 제외',
'84': '공공 행정, 국방 및 사회보장 행정',
'85': '교육 서비스업',
'86': '보건업',
'87': '사회복지 서비스업',
'90': '창작, 예술 및 여가관련 서비스업',
'91': '스포츠 및 오락관련 서비스업',
'94': '협회 및 단체',
'95': '개인 및 소비용품 수리업',
'96': '기타 개인 서비스업',
'97': '가구 내 고용활동',
'98': '달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동',
'99': '국제 및 외국기관'
}

In [109]:
dict_to_ind_02 = {}
for idx, value in ind_02_dict.items():
    dict_to_ind_02[value] = idx
dict_to_ind_02

{'농업': '01',
 '임업': '02',
 '어업': '03',
 '석탄, 원유 및 천연가스 광업': '05',
 '금속 광업': '06',
 '비금속광물 광업; 연료용 제외': '07',
 '광업 지원 서비스업': '08',
 '식료품 제조업': '10',
 '음료 제조업': '11',
 '담배 제조업': '12',
 '섬유제품 제조업; 의복 제외': '13',
 '의복, 의복 액세서리 및 모피제품 제조업': '14',
 '가죽, 가방 및 신발 제조업': '15',
 '목재 및 나무제품 제조업; 가구 제외': '16',
 '펄프, 종이 및 종이제품 제조업': '17',
 '인쇄 및 기록매체 복제업': '18',
 '코크스, 연탄 및 석유정제품 제조업': '19',
 '화학 물질 및 화학제품 제조업; 의약품 제외': '20',
 '의료용 물질 및 의약품 제조업': '21',
 '고무 및 플라스틱제품 제조업': '22',
 '비금속 광물제품 제조업': '23',
 '1차 금속 제조업': '24',
 '금속 가공제품 제조업; 기계 및 가구 제외': '25',
 '전자 부품, 컴퓨터, 영상, 음향 및 통신장비 제조업': '26',
 '의료, 정밀, 광학 기기 및 시계 제조업': '27',
 '전기장비 제조업': '28',
 '기타 기계 및 장비 제조업': '29',
 '자동차 및 트레일러 제조업': '30',
 '기타 운송장비 제조업': '31',
 '가구 제조업': '32',
 '기타 제품 제조업': '33',
 '산업용 기계 및 장비 수리업': '34',
 '전기, 가스, 증기 및 공기 조절 공급업': '35',
 '수도업': '36',
 '하수, 폐수 및 분뇨 처리업': '37',
 '폐기물 수집, 운반, 처리 및 원료 재생업': '38',
 '환경 정화 및 복원업': '39',
 '종합 건설업': '41',
 '전문직별 공사업': '42',
 '자동차 및 부품 판매업': '45',
 '도매 및 상품 중개업': '46',
 '소매업; 자동차 제외': '4

In [84]:
ind_to_stock_dict = {'01' : '음식료품',
'02' : '음식료품',
'03' : '음식료품',
'05': '비금속광물',
'06': '철강금속',
'07': '비금속광물',
'08': '서비스업',
'10': '음식료품',
'11': '음식료품',
'12': '기타',
'13': '섬유의복',
'14': '섬유의복',
'15': '섬유의복',
'16': '종이목재',
'17': '종이목재',
'18': '서비스업',
'19': '화학',
'20': '화학',
'21': '의약품',
'22': '화학',
'23': '비금속광물',
'24': '철강금속',
'25': '철강금속',
'26': '전기전자',
'27': '의료정밀',
'28': '전기장비 제조업',
'29': '기계',
'30': '운수장비',
'31': '운수장비',
'32': '가구 제조업',
'33': '기타',
'34': '기타',
'35': '전기가스업',
'36': '기타',
'37': '기타',
'38': '기타',
'39': '기타',
'41': '건설업',
'42': '건설업',
'45': '운수장비',
'46': '유통업',
'47': '유통업',
'49': '유통업',
'50': '유통업',
'51': '유통업',
'52': '운수창고',
'55': '서비스업',
'56': '서비스업',
'58': '서비스업',
'59': '서비스업',
'60': '통신업',
'61': '통신업',
'62': '서비스업',
'63': '통신업',
'64': '금융업',
'65': '보험',
'66': '금융업',
'68': '기타',
'70': '연구개발업',
'71': '서비스업',
'72': '기타',
'73': '서비스업',
'74': '서비스업',
'75': '서비스업',
'76': '서비스업',
'84': '기타',
'85': '서비스업',
'86': '보건업',
'87': '서비스업',
'90': '서비스업',
'91': '서비스업',
'94': '협회 및 단체',
'95': '기타',
'96': '서비스업',
'97': '기타',
'98': '기타',
'99': '기타'
}

In [85]:
# 기타 :
'''
'12': '담배 제조업',
'32': '가구 제조업',
'33': '기타 제품 제조업',
'36': '수도업',
'37': '하수, 폐수 및 분뇨 처리업',
'38': '폐기물 수집, 운반, 처리 및 원료 재생업',
'39': '환경 정화 및 복원업',
'68': '부동산업',
'72': '건축 기술, 엔지니어링 및 기타 과학기술 서비스업',
'84': '공공 행정, 국방 및 사회보장 행정',
'94': '협회 및 단체',
'97': '가구 내 고용활동',
'98': '달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동',
'99': '국제 및 외국기관'
'''

"\n'12': '담배 제조업',\n'32': '가구 제조업',\n'33': '기타 제품 제조업',\n'36': '수도업',\n'37': '하수, 폐수 및 분뇨 처리업',\n'38': '폐기물 수집, 운반, 처리 및 원료 재생업',\n'39': '환경 정화 및 복원업',\n'68': '부동산업',\n'72': '건축 기술, 엔지니어링 및 기타 과학기술 서비스업',\n'84': '공공 행정, 국방 및 사회보장 행정',\n'94': '협회 및 단체',\n'97': '가구 내 고용활동',\n'98': '달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동',\n'99': '국제 및 외국기관'\n"

In [89]:
nb_active_ind.ind_02_cate.isna().sum()

62

In [106]:
# del nb_active_ind['Unnamed: 0']
act_ind_drop = nb_active_ind.dropna(subset='ind_02_cate')
act_ind_drop.ind_02_cate.isna().sum()

0

In [117]:
act_ind_drop['stock_ind'] = list(map((lambda x : ind_to_stock_dict[dict_to_ind_02[x]]), act_ind_drop['ind_02_cate']))
act_ind_drop

C:\Users\Admin\AppData\Local\Temp\ipykernel_20924\2854100286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  act_ind_drop['stock_ind'] = list(map((lambda x : ind_to_stock_dict[dict_to_ind_02[x]]), act_ind_drop['ind_02_cate']))


,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_TYP,CMP_SCL,PBCO_GB,...,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,ind_01_cate,ind_02_cate,stock_ind
0,2218116923,NaN,NaN,신우종합건설,(주),"Shinu General Construction Co., Ltd.",M,4,2,2,...,1.0,NaN,토목공사업,NaN,NaN,김주대외 1명,12.0,건설업,종합 건설업,건설업
1,1138111739,NaN,한일가스산업(주),한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,3,2,2,...,1.0,NaN,LP Gas 충전 및 판매업,NaN,www.hanilgas.com,홍순철외 1명,43.0,도매 및 소매업,소매업; 자동차 제외,유통업
2,1108146490,(주),NaN,랩솔루션,NaN,Lab Solution,M,4,2,2,...,1.0,NaN,기체분석기 등,NaN,www.lab-solution.com,김봉철,3.0,제조업,"의료, 정밀, 광학 기기 및 시계 제조업",의료정밀
3,3128134737,(주),NaN,고려인삼제품공사,NaN,"KOREAN GINSENG PRODUCT CO., LTD",M,4,2,2,...,1.0,NaN,"다류, 차, 인삼제품, 음료, 코코아가공품, 커피",NaN,www.kitea.co.kr,박경출,24.0,제조업,식료품 제조업,음식료품
4,2078132193,(주),(주)엠피아이,엠피아이,NaN,"EMPIALEE CO.,LTD.",M,3,2,2,...,1.0,NaN,가죽 및 대용가죽제품 제조업,NaN,www.mastercnd.co.kr,이스테판상수외 1명,35.0,제조업,"가죽, 가방 및 신발 제조업",섬유의복
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198645,6238102213,(주),NaN,준영,NaN,"Junyoung Co., Ltd.",M,4,2,2,...,1.0,NaN,"비금속류 원료 재생업, 폐플라스틱 가공처리",NaN,NaN,맹성호,NaN,"수도, 하수 및 폐기물 처리, 원료 재생업","폐기물 수집, 운반, 처리 및 원료 재생업",기타
198646,6208701973,(주),NaN,동화피앤피,NaN,"Donghwa P&P Co., Ltd.",M,4,2,2,...,1.0,NaN,"인쇄, 지함, 코팅, 디자인",NaN,NaN,홍종오,NaN,제조업,인쇄 및 기록매체 복제업,서비스업
198647,3688702720,NaN,NaN,중앙종합조경,(주),NaN,M,4,2,2,...,1.0,NaN,"조경공사, 토목공사",NaN,NaN,서영희,13.0,건설업,종합 건설업,건설업
198648,4868602701,(주),NaN,세움웹,NaN,NaN,M,5,2,2,...,1.0,NaN,"소프트웨어 자문, 개발 및 공급업",NaN,NaN,이대홍,NaN,정보통신업,출판업,서비스업


# 주식 업종별 평균 지수 크롤링 데이터 만들기
1. 코스피 기준 각 업종별 시총 가장 높은 5개 주식 가져오기 (다음금융 기준)
2. 해당 주식의 주가를 기록하여 데이터에 입력 (인덱스를 날짜값으로) + 코스피 기준
3. 해당업종의 평균 코스피 대비 주가지수 비율을 계산
4. minmax스케일링 하여 적용
-> 오늘은 각 산업별 시가총액 기준 상위 5개 기업 주가크롤링 코드 까지 작성

In [196]:
# 클릭
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
driver.find_element(By.CSS_SELECTOR, '#boxStocks > div.box_contents > div > div > a:nth-child(2)').click()

In [56]:
driver = webdriver.Chrome(r'C:\pydata\chromedriver.exe')
final_ind_list = []

# 각 업종별 하이퍼링크 가져오기
url = 'https://finance.daum.net/domestic/sectors'
driver.get(url)
time.sleep(2)
html = driver.page_source
soup = bs(html, 'html.parser')
href_list = []
not_list = ['시가총액규모대', '코스피 지수', '시가총액규모중', '시가총액규모소']
for tr in soup.select_one('tbody').select('th'):
        ind = tr.text
        if ind in not_list :
            pass
        else :
            try :
                href = tr.select_one('a')['href']
                href = 'https://finance.daum.net' + href
                href_list.append([ind, href])
            except :
                pass

# 업종별 시가총액 기준 상위 5개 링크 및 업종코드 불러오기
print('링크 리스트 :', '\n',href_list)
for hrf in href_list:
    url = hrf[1]
    driver.get(url)
    driver.implicitly_wait(10)
    time.sleep(1)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    stock_list = []
    pg_list = soup.select_one('.paging').select('a')
    pg_num = len(pg_list)
    # 1페이지 까지 있는 경우
    if pg_num == 0:
        html = driver.page_source
        soup = bs(html, 'html.parser')
        for tr in soup.select_one('tbody').select('tr'):
            ind_name = tr.select_one('th').text
            ind_total = int(tr.select('td')[-2].text.strip().replace(',',''))
            href = tr.select_one('th').select_one('a')['href']
            href = 'https://finance.daum.net' + href
            stock_list.append([ind_name, ind_total, href, href[-6:]])
    else :
    # 다중 페이지 클릭 코드
        for pg in range(0, pg_num+1) :
            if pg == 0 :
                pass
            else :
                css_path = f'#boxStocks > div.box_contents > div > div > a:nth-child({pg + 1})'
                driver.find_element(By.CSS_SELECTOR, css_path).click()
                driver.implicitly_wait(10)
                time.sleep(0.5)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            for tr in soup.select_one('tbody').select('tr'):
                ind_name = tr.select_one('th').text
                ind_total = int(tr.select('td')[-2].text.strip().replace(',',''))
                href = tr.select_one('th').select_one('a')['href']
                href = 'https://finance.daum.net' + href
                stock_list.append([ind_name, ind_total, href, href[-6:]])
    df_stcok = pd.DataFrame(stock_list, columns=['stcok', 'total', 'href', 'code'])
    top_5 = df_stcok.sort_values(by='total', ascending=False).head().stcok.values
    top_5_href = df_stcok.sort_values(by='total', ascending=False).head().href.values
    top_5_code = df_stcok.sort_values(by='total', ascending=False).head().code.values
    final_ind_list.append([hrf[0], list(top_5), list(top_5_href), list(top_5_code)])


final_ind_list

C:\Users\Admin\AppData\Local\Temp\ipykernel_19792\2529293260.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\pydata\chromedriver.exe')


링크 리스트 : [['의약품', 'https://finance.daum.net/domestic/sectors/D0011009'], ['전기전자', 'https://finance.daum.net/domestic/sectors/D0011013'], ['비금속광물', 'https://finance.daum.net/domestic/sectors/D0011010'], ['통신업', 'https://finance.daum.net/domestic/sectors/D0011020'], ['음식료품', 'https://finance.daum.net/domestic/sectors/D0011005'], ['종이목재', 'https://finance.daum.net/domestic/sectors/D0011007'], ['은행', 'https://finance.daum.net/domestic/sectors/D0011022'], ['서비스업', 'https://finance.daum.net/domestic/sectors/D0011026'], ['전기가스업', 'https://finance.daum.net/domestic/sectors/D0011017'], ['운수장비', 'https://finance.daum.net/domestic/sectors/D0011015'], ['보험', 'https://finance.daum.net/domestic/sectors/D0011025'], ['의료정밀', 'https://finance.daum.net/domestic/sectors/D0011014'], ['섬유의복', 'https://finance.daum.net/domestic/sectors/D0011006'], ['금융업', 'https://finance.daum.net/domestic/sectors/D0011021'], ['증권', 'https://finance.daum.net/domestic/sectors/D0011024'], ['운수창고', 'https://finance.daum.net/do

[['의약품',
  ['삼성바이오로직스', '셀트리온', 'SK바이오사이언스', '유한양행', '한미약품'],
  ['https://finance.daum.net/quotes/A207940',
   'https://finance.daum.net/quotes/A068270',
   'https://finance.daum.net/quotes/A302440',
   'https://finance.daum.net/quotes/A000100',
   'https://finance.daum.net/quotes/A128940'],
  ['207940', '068270', '302440', '000100', '128940']],
 ['전기전자',
  ['삼성전자', 'LG에너지솔루션', 'SK하이닉스', '삼성SDI', '삼성전자우'],
  ['https://finance.daum.net/quotes/A005930',
   'https://finance.daum.net/quotes/A373220',
   'https://finance.daum.net/quotes/A000660',
   'https://finance.daum.net/quotes/A006400',
   'https://finance.daum.net/quotes/A005935'],
  ['005930', '373220', '000660', '006400', '005935']],
 ['비금속광물',
  ['포스코케미칼', '쌍용C&E', '아이에스동서', '한일시멘트', 'KCC글라스'],
  ['https://finance.daum.net/quotes/A003670',
   'https://finance.daum.net/quotes/A003410',
   'https://finance.daum.net/quotes/A010780',
   'https://finance.daum.net/quotes/A300720',
   'https://finance.daum.net/quotes/A344820'],
  ['003670

In [57]:
df_top = pd.DataFrame(final_ind_list, columns=['ind', 'top_5_ind_list', 'top_5_href', 'top_5_code'])
df_top

,ind,top_5_ind_list,top_5_href,top_5_code
0,의약품,"[삼성바이오로직스, 셀트리온, SK바이오사이언스, 유한양행, 한미약품]","[https://finance.daum.net/quotes/A207940, http...","[207940, 068270, 302440, 000100, 128940]"
1,전기전자,"[삼성전자, LG에너지솔루션, SK하이닉스, 삼성SDI, 삼성전자우]","[https://finance.daum.net/quotes/A005930, http...","[005930, 373220, 000660, 006400, 005935]"
2,비금속광물,"[포스코케미칼, 쌍용C&E, 아이에스동서, 한일시멘트, KCC글라스]","[https://finance.daum.net/quotes/A003670, http...","[003670, 003410, 010780, 300720, 344820]"
3,통신업,"[SK텔레콤, KT, LG유플러스, 현대퓨처넷, 인스코비]","[https://finance.daum.net/quotes/A017670, http...","[017670, 030200, 032640, 126560, 006490]"
4,음식료품,"[CJ제일제당, 오리온, 농심, 오뚜기, 하이트진로]","[https://finance.daum.net/quotes/A097950, http...","[097950, 271560, 004370, 007310, 000080]"
5,종이목재,"[한솔제지, 아세아제지, 신대양제지, 무림P&P, 영풍제지]","[https://finance.daum.net/quotes/A213500, http...","[213500, 002310, 016590, 009580, 006740]"
6,은행,[],[],[]
7,서비스업,"[NAVER, 카카오, 삼성에스디에스, 크래프톤, 엔씨소프트]","[https://finance.daum.net/quotes/A035420, http...","[035420, 035720, 018260, 259960, 036570]"
8,전기가스업,"[한국전력, 한국가스공사, 서울가스, 삼천리, SGC에너지]","[https://finance.daum.net/quotes/A015760, http...","[015760, 036460, 017390, 004690, 005090]"
9,운수장비,"[현대차, 기아, 현대모비스, 현대중공업, 한국조선해양]","[https://finance.daum.net/quotes/A005380, http...","[005380, 000270, 012330, 329180, 009540]"


In [58]:
df_top = pd.DataFrame(final_ind_list, columns=['ind', 'top_5_ind_list', 'top_5_href', 'top_5_code'])
df_top.to_csv('업종별_href_code_list_1025.csv', index=False)